In [1]:
!rm -rf minsearch.py
!wget https://raw.githubusercontent.com/yakhyo/llm-practice/main/01-intro/minsearch.py

--2024-07-04 14:44:56--  https://raw.githubusercontent.com/yakhyo/llm-practice/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3889 (3.8K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.80K  --.-KB/s    in 0s      

2024-07-04 14:44:57 (75.1 MB/s) - ‘minsearch.py’ saved [3889/3889]



In [2]:
import requests
import minsearch

docs_url = (
    "https://github.com/yakhyo/llm-practice/blob/main/01-intro/documents.json?raw=1"
)
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    course_name = course["course"]

    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

In [3]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

## Create an search index and parse all the documents to it

In [4]:
index = minsearch.Index(
    text_fields=["question", "text", "section"], keyword_fields=["course"]
)

index.fit(documents)

In [5]:
def search_query(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5,
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt


def llm(prompt):
    response = client.chat.completions.create(
        model="phi3", messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [7]:
def rag_system(query):
    search_results = search_query(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [13]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama",
)

In [15]:
a = rag_system("I just discovered the course. Can I still join?")

In [16]:
print(_)

 Yes, even though I have not registered yet, as long as the final project deadlines are met, which usually occur after office hours and at least one week into the course starting on January 15th, 2024, there seems to be some flexibility for submissions throughout the duration of the course. Please note that it's important not to leave all tasks until the final day as this can disrupt your learning experience due to tight deadlines typically set after office hours or at least one week into the class sessions starting on January 15th, 2024.
